In [ ]:
import pandas as pd
import numpy as np
import random
import itertools
import tqdm

In [ ]:
def create_data(n, type):
    # job 갯수
    num_jobs = n
    # 균등분포 데이터 만들기
    def generate_input_data_uniform(num_jobs):
        data = []
        for job_id in range(1, num_jobs + 1):
            while True:
                release_time = 0 # 출제시간
                processing_time = int(np.random.uniform(1, 15)) # 소요시간 # 최소1, 최대10 균등분포
                due_date = int(np.random.uniform(release_time + processing_time, 40)) # 제출기한
                if  processing_time < due_date: # 제약조건
                    break
            rate = random.randint(1, 3) # 성적 반영 비율
            data.append((job_id, release_time, processing_time, due_date, rate))
        return data
    # 정규분포 데이터 만들기
    def generate_input_data_normal(num_jobs):
        data = []
        for job_id in range(1, num_jobs + 1):
            while True:
                processing_time = int(max(0, np.random.normal(10, 1))) # 평균5, 표준편차2 정규분포
                release_time = 0
                #release_time = int(max(0, np.random.normal(1, processing_time)) )
                due_date = int(max(release_time+processing_time, np.random.normal(8, 2)))
                if release_time < processing_time and processing_time < due_date:
                    break
            rate = random.randint(1, 3)
            data.append((job_id, release_time, processing_time, due_date, rate))
        return data
    
    if type == 'uniform':
        output_data = generate_input_data_uniform(num_jobs) # 균일분포로 데이터 생성
    else:
        output_data = generate_input_data_normal(num_jobs) # 정규분포로 데이터 생성

    df = pd.DataFrame(output_data, columns=['job', '출제시간', '소요시간', '제출기한', '성적 반영비율'])
    df = df.set_index('job').transpose()
    df.columns = ['job' + str(i) for i in range(1, num_jobs + 1)]
    df.index = ['출제시간', '소요시간', '제출기한', '성적 반영비율']

    filename = f'{num_jobs}_job_{type} data.csv'
    df.to_csv(filename, index=True, encoding='utf-8-sig')

    print(f"{num_jobs}개 job {type} data 생성.")

    return num_jobs, type

In [ ]:
# num_job, type = create_data(100, 'uniform')

In [ ]:
#output_file = f'{num_job}_job_{type} data.csv'
output_file = '10_job_normal data.csv'
input_data = pd.read_csv(output_file, index_col=0)
df = input_data

objective_list = ['total_flowtime', 'makespan', 'tardy_job', 'total_tardiness', 'total_weighted_tardiness']
object = objective_list[3]

def get_fitness(sequence):
    flowtime = 0
    total_flowtime = 0
    makespan = 0
    tardiness = 0
    total_tardiness = 0
    tardy_job = 0
    total_weighted_tardiness = 0
    

    for i in sequence:
        job = df['job' + str(i)]
        flowtime += job['출제시간'] + job['소요시간']
        total_flowtime += flowtime

        makespan = flowtime
        
        if flowtime - job['제출기한'] >= 0:
            tardiness = flowtime - job['제출기한']
            tardy_job += 1
            total_tardiness += tardiness
        else:
            tardiness = 0

        weighted_tardiness = job['성적 반영비율'] * tardiness
        total_weighted_tardiness += weighted_tardiness
        
    ob_list = {'total_flowtime':total_flowtime, 'makespan':makespan, 'tardy_job':tardy_job,'total_tardiness':total_tardiness, 'total_weighted_tardiness':total_weighted_tardiness}
    return ob_list[object]
    

results = get_fitness([1,2,3,4,5,6,7,8,9,10])

print('{} : {}'.format(objective_list[3],results))

In [6]:

#output_file = f'{num_job}_job_{type} data.csv'
output_file = '100_job_uniform data.csv'
input_data = pd.read_csv(output_file, index_col=0)
df = input_data

def full_enumeration(object):
    num_jobs = len(df.columns)
    best_sequence = None
    best_objective = float('inf')
    best_set = []

    # 모든 가능한 순열 조합 생성
    permutations = list(itertools.permutations(range(1, num_jobs + 1), num_jobs))
    print(f'full_enumerate 갯수 {num_jobs}! :', len(permutations))
    
    for sequence in tqdm(permutations, desc='Calculating', unit='sequence'):
        objective = get_fitness(sequence)

        if objective < best_objective:
            best_objective = objective
            best_sequence = sequence
            best_set = [[best_sequence, best_objective]]
            
        elif objective == best_objective and sequence != best_sequence:
            best_sequence = sequence
            best_set.append([best_sequence, best_objective])

    return object, best_sequence, best_objective, best_set


object, best_sequence, best_objective, best_set = full_enumeration('total_tardiness')

print(f'{object} 최소화')
print(f'Best sequence: {best_sequence}')
print(f'Best {object}: {best_objective}')
print(f'{object} 최소화 가능한 모든 sequence', best_set)

In [ ]:
import pandas as pd 

df = pd.read_csv("100_job_uniform data.csv", index_col=0)
p_time_sort = df.sort_values(by='소요시간', axis=1)

flowtime = 0
total_flowtime = 0
for i in p_time_sort:
    job = df[i]
    flowtime += job['소요시간']
    total_flowtime += flowtime

total_flowtime

In [49]:
import pandas as pd

output_file = '100_job_uniform data.csv'
#output_file = '10_job_normal data.csv'
input_data = pd.read_csv(output_file, index_col=0)
df = input_data
objective_list = ['total_flowtime', 'makespan', 'tardy_job', 'total_tardiness', 'total_weighted_tardiness']
object = objective_list[3]

def get_fitness(sequence):
    flowtime = 0
    total_flowtime = 0
    makespan = 0
    tardiness = 0
    total_tardiness = 0
    tardy_job = 0
    total_weighted_tardiness = 0
    

    for i in sequence:
        job = df['job' + str(i)]
        flowtime += job['출제시간'] + job['소요시간']
        total_flowtime += flowtime

        makespan = flowtime
        
        if flowtime - job['제출기한'] >= 0:
            tardiness = flowtime - job['제출기한']
            tardy_job += 1
            total_tardiness += tardiness
        else:
            tardiness = 0

        weighted_tardiness = job['성적 반영비율'] * tardiness
        total_weighted_tardiness += weighted_tardiness
        
    ob_list = {'total_flowtime':total_flowtime, 'makespan':makespan, 'tardy_job':tardy_job,'total_tardiness':total_tardiness, 'total_weighted_tardiness':total_weighted_tardiness}
    return ob_list[object]

def generate_permutations(sequence, current_index, permutations):
    if current_index == len(sequence) - 1:
        perm = get_fitness(sequence)
        if permutations[1] > perm:
            permutations[0] = sequence[:]
            permutations[1] = perm
            print(perm)
    else:
        for i in range(current_index, len(sequence)):
            sequence[current_index], sequence[i] = sequence[i], sequence[current_index]
            generate_permutations(sequence, current_index + 1, permutations)
            sequence[current_index], sequence[i] = sequence[i], sequence[current_index]
    return permutations

num_jobs = 100
sequence = list(range(1, num_jobs + 1))
permutations = [sequence, get_fitness(sequence)]

permutations = generate_permutations(sequence, 0, permutations)
print('결과' , permutations)
# print(f'Best sequence: {permutations[0][0]}')
# print(f'Best objective: {permutations[0][1]}') # 39441

39627
39625
39615
39607
39600
39598
39594
39592
39589
39587
39582
39580
39578
39575
39573
39571
39570
39568
39567
39565
39564
39563
39561
39558
39557
39554
39552
39551
39550
39545
39543
39538
39536
39534


KeyboardInterrupt: 